#Diffuser Stable Diffusion Jup
A first implementation of semi complete tools about SD 1.5 and SDXL on jupyter-colab</br>
##Guide:</br>
<div>
<ol>
<li>Launch Installation requirements at beginning</li>
<li>Launch Environment prepare after the previous step is ended and the instance has auto restarted</li>
<li>Launch the Path selector</li>
<li>Model selector
<ol>
•Download the model (it use Civitai)
•Select the model
</ol></li>
<li>Possibile Adapters only SD 1.5</li>
<li>Generate the images</li>
<li>Detailer on generated images</li>
</ol>
</div>
The implementation use a drive as storage.</br>

##Features</br>

<li>Text/IMG-2img  SD 1.5 and SDXL</li>
<li>Multiple LORAS</li>
<li>Single embedding model</li>
<li>single controlnet SD 1.5</li>
<li>basic detailer SD 1.5 and SDXL</li>
</br>
</br>
<a target="_blank" href="https://colab.research.google.com/github/chri002/sd_1.5_diffuser/blob/main/sd_jup_diffuser.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" height="30px"/>
</a>

#Installation requirements
This block restarts automatically, so don't start any other block before it finished and DON'T PUSH REBOOT!!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Check code env
from pathlib import Path
import os

HOME = "/content/drive/MyDrive/"
COMF = HOME + "/sd_1.5_diffuser"
IP_ADAPTER = COMF + "/IP-Adapter"



if not(os.path.exists(COMF)):
   %cd {HOME}
   !git clone https://github.com/chri002/sd_1.5_diffuser

if not(os.path.exists(IP_ADAPTER)):
   %cd {COMF}
   !git clone https://github.com/tencent-ailab/IP-Adapter
%cd "/content/"


!pip install -r {COMF+"/requirements.txt"}

In [ ]:
!pip install git+https://github.com/huggingface/diffusers -q

In [ ]:
import os
os.kill(os.getpid(), 9)

#Enviroment prepare
Lunch and accept drive connection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import sys

HOME = "/content/drive/MyDrive/"
COMF = HOME + "/sd_1.5_diffuser"
IP_ADAPTER = COMF + "/IP-Adapter"

sys.path.insert(0,COMF)
sys.path.insert(1,IP_ADAPTER)

import random
import mediapy as media
import torch
import numpy as np
from torch import autocast

from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, AutoencoderTiny, StableDiffusionXLPipeline, DiffusionPipeline
from PIL import Image, ImageDraw, ImageFilter, ImageFile
from PIL.PngImagePlugin import PngInfo
import cv2
import diffusers

import ipywidgets as widgets
from IPython.display import display, clear_output
import torch.nn.functional as F

from diffusers.models.attention_processor import (
      AttnProcessor,
      AttnProcessor2_0,
  )

from controlnet_aux.processor import Processor

from importlib import reload
import lpw_stable_diffusion
import lpw_stable_diffusion_xl
import ADetailer
from safetensors_file import SafeTensorsFile

import threading
from uNet2DConditionModelFree import *
import json

from insightface.app import FaceAnalysis
from insightface.utils import face_align

from diffusers.models.attention_processor import AttnProcessor2_0
from diffusers import AutoencoderKL, DEISMultistepScheduler, LMSDiscreteScheduler, KDPM2DiscreteScheduler, PNDMScheduler, DPMSolverMultistepScheduler,KDPM2AncestralDiscreteScheduler, UniPCMultistepScheduler, EulerDiscreteScheduler, DDPMScheduler, EulerAncestralDiscreteScheduler, DPMSolverSinglestepScheduler,DDIMScheduler, HeunDiscreteScheduler

In [ ]:
def path(a,ck=True,lora=False,emb=False,cont=False,vae=False):
  if(ck):
    return model_path+"/"+a
  else:
   if lora:
    return lora_path+"/"+a
   elif cont:
    return controlnet_path+"/"+a
   elif emb:
    return embedding_path+"/"+a
   else:
    return vaes_path+"/"+a

In [ ]:
import gc
def fresh():
  gc.collect()
  if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()

In [ ]:
def getLoras(path):
    loras = {}

    for path, sub_folder, files in os.walk(path):
        for folder in sub_folder:
            if not(".ipynb_checkpoints" in folder):
               loras[folder] = {}
        fol=len(loras.keys())

        files.sort()
        for elem in files:
           if elem.endswith("safetensors") or elem.endswith("pt"):

              if fol>0:
                loras[path.split("/")[-1]][elem.split(".")[0]] = {"name_file":elem, "path":path}
              else:
                loras[elem.split(".")[0]] = {"name_file":elem, "path":path}

    return loras

def get_tokens(input_file:str) -> int:
    s=SafeTensorsFile.open_file(input_file)
    js=s.get_header()
    if js is None: return -1

    tags = []
    if "__metadata__" in js.keys():
      if "ss_tag_frequency" in js["__metadata__"].keys():
        for elem in json.loads(js["__metadata__"]["ss_tag_frequency"]):
          tags += ([str(x) for x in (json.loads(js["__metadata__"]["ss_tag_frequency"])[elem].keys())])
    return tags[:10]

In [ ]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
def prepareImage(path, app0):

  #app0 = FaceAnalysis(name="antelopev2", providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
  app0.prepare(ctx_id=0, det_size=(640, 640))

  image = cv2.imread(path)#"A.jpeg")
  im = image
  faceid_embeds=None
  for size in [(size,size) for size in range(768,128,-64)]:
    app0.det_model.det_size=size
    print("detection size:"+str(size))
    im = image
    for sizei in [(sizex,sizey)  for sizex in range(640, 448, -64) for sizey in range(640,448,-64)]:

     im = cv2.resize(im, sizei)
     faces = app0.get(im)
     if len(faces)>0:



      print("face reconize:"+str(sizei))
      faceid_embeds =(torch.from_numpy(faces[0].normed_embedding).unsqueeze(0))
      face_image = (face_align.norm_crop(im, landmark=faces[0].kps, image_size=512)) # you can also segment the fac

      break



     #faces = app0.get(im)

    if(faceid_embeds!=None):
      break
  print(size, im.shape)
  return faceid_embeds , face_image

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
half_value = True
vae_tiny = None
sdxl = False

def vae_load():
  global vae_tiny
  vae_tiny_t = AutoencoderTiny.from_pretrained("madebyollin/taesd" if not(sdxl) else "madebyollin/taesdxl", torch_dtype=torch.float16 if half_value else torch.float32)
  vae_tiny = vae_tiny_t.to("cuda")
  del vae_tiny_t

last_thread = None

In [ ]:


def get_pipeline_embeds(pipeline, prompt, negative_prompt, device):

    max_length = pipeline.tokenizer.model_max_length

    print(max_length)

    # simple way to determine length of tokens
    count_prompt = (pipeline.tokenizer(prompt, return_tensors="pt", truncation=False).input_ids).shape[-1]
    count_negative_prompt = (pipeline.tokenizer(negative_prompt, return_tensors="pt", truncation=False).input_ids).shape[-1]



    # create the tensor based on which prompt is longer
    if count_prompt >= count_negative_prompt:
        input_ids = pipeline.tokenizer(prompt, return_tensors="pt", truncation=False).input_ids.to(device)
        shape_max_length = input_ids.shape[-1]
        negative_ids = pipeline.tokenizer(negative_prompt, truncation=False, padding="max_length",
                                          max_length=shape_max_length, return_tensors="pt").input_ids.to(device)

        print("  length of prompt:\t"+str(shape_max_length))

    else:
        negative_ids = pipeline.tokenizer(negative_prompt, return_tensors="pt", truncation=False).input_ids.to(device)
        shape_max_length = negative_ids.shape[-1]
        input_ids = pipeline.tokenizer(prompt, return_tensors="pt", truncation=False, padding="max_length",
                                       max_length=shape_max_length).input_ids.to(device)
        print("  length of prompt:\t"+str(shape_max_length))

    concat_embeds = []
    neg_embeds = []
    for i in range(0, shape_max_length, max_length):
        concat_embeds.append(pipeline.text_encoder(input_ids[:, i: i + max_length])[0])
        neg_embeds.append(pipeline.text_encoder(negative_ids[:, i: i + max_length])[0])

    return torch.cat(concat_embeds, dim=1), torch.cat(neg_embeds, dim=1)

def modifyScheduler(pipeline,scheduler):
   if(scheduler=='DDPMScheduler'):
     return DDPMScheduler.from_config(pipeline["model"].scheduler.config)
   elif(scheduler=='DEISMultistepScheduler'):
     return DEISMultistepScheduler.from_config(pipeline["model"].scheduler.config)
   elif(scheduler=='DPMSolverMultistepScheduler'):
     return DPMSolverMultistepScheduler.from_config(pipeline["model"].scheduler.config)
   elif(scheduler=='DPMSolverSinglestepScheduler'):
     return DPMSolverSinglestepScheduler.from_config(pipeline["model"].scheduler.config)
   elif(scheduler=='EulerAncestralDiscreteScheduler'):
     return EulerAncestralDiscreteScheduler.from_config(pipeline["model"].scheduler.config)
   elif(scheduler=='EulerDiscreteScheduler'):
     return EulerDiscreteScheduler.from_config(pipeline["model"].scheduler.config)
   elif(scheduler=='HeunDiscreteScheduler'):
     return HeunDiscreteScheduler.from_config(pipeline["model"].scheduler.config)
   elif(scheduler=='KDPM2AncestralDiscreteScheduler'):
     return KDPM2AncestralDiscreteScheduler.from_config(pipeline["model"].scheduler.config)
   elif(scheduler=='KDPM2DiscreteScheduler'):
     return KDPM2DiscreteScheduler.from_config(pipeline["model"].scheduler.config)
   elif(scheduler=='LMSDiscreteScheduler'):
     return LMSDiscreteScheduler.from_config(pipeline["model"].scheduler.config)
   elif(scheduler=='PNDMScheduler'):
     return PNDMScheduler.from_config(pipeline["model"].scheduler.config)
   else:
     return UniPCMultistepScheduler.from_config(pipeline["model"].scheduler.config)



def image_grid(imgs, rows, cols):

    w, h = imgs[0].size

    ll = len(imgs)

    while(rows*cols<ll):
      rows+=1

    grid = Image.new('RGBA', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size



    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid


def generate(pipe, args):

  gc.collect()
  torch.cuda.empty_cache()

  image = []
  seeds = 0
  if args["seed"]==None:
    seeds = [random.randint(-int(sys.maxsize),int(sys.maxsize))]
  else:
    seeds = args["seed"]

  ImaTot = []
  width = int(args["width"] //8)*8
  height= int(args["height"] //8)*8



  for _ in range(args["batch_number"]):
    generator = torch.Generator(device).manual_seed(seeds[-1])
    seeds.append(seeds[-1]+1)
    for (idE,pr) in enumerate([[args["prompt"]]]):
      pr_str  = (''.join(pr))
      neg_str = (''.join([[args["negPrompt"]]][idE]))
      step_str = ''.join([str(stp) for stp in [args["step"]]])
      print("Image generator:\n  positive prompt:\t"+pr_str+"\n  negative prompt:\t"+neg_str+"\n  number of steps:\t"+step_str+"\n  size image:\t\t("+str(width)+","+str(height)+")\n  seed value:\t\t"+str(seeds[-1])+"\n  strength generator:\t"+str(args["strength"]))
      image = []
      for step in [args["step"]]:
        gc.collect()
        torch.cuda.empty_cache()

        generator = torch.Generator(device).manual_seed(seeds[-1])

        pr = pr * (args["batch_size"]  if not("sdxl" in args ) else 1)
        negPr = ""
        if not("sdxl" in args ):
          negPr = [[args["negPrompt"]]][idE] * args["batch_size"]
        else:
          negPr = [[args["negPrompt"]]][idE]

        init_image = args["init_image"].convert("RGB")


        mask_image = args["mask_image"].convert("RGB")


        if "cb" not in args.keys():
          args["cb"] = None

        with torch.inference_mode():
          if "image" in args["Pp"]:
              image.append(pipe.img2img(prompt=pr, negative_prompt=negPr, guidance_scale=args["strength"], generator=generator, num_inference_steps=step,
                                        width=width, height=height, image=init_image, strength=args["imageStrength"],
                                        max_embeddings_multiples=args["mxl"], callback=args["cb"], **args["sdxl"]).images)
          elif "text" in args["Pp"]:
              image.append(pipe.text2img(prompt=pr, negative_prompt=negPr, guidance_scale=args["strength"], generator=generator, num_inference_steps=step,
                                         width=width, height=height, max_embeddings_multiples=args["mxl"], callback=args["cb"],
                                         controlnet_conditioning_scale=args["controlnet_strenght"],
                                         image_controlnet=args["controlnet_image"], control_guidance_start=args["start_cnt"],
                                         control_guidance_end=args["end_cnt"], **args["sdxl"]).images)
          else:
              print("inpaint")
              display(mask_image)
              image.append(pipe.inpaint(prompt=pr, negative_prompt=negPr, guidance_scale=args["strength"], generator=generator, num_inference_steps=step,
                                        width=width, height=height, image=(init_image), mask_image=mask_image,
                                        strength=args["imageStrength"], max_embeddings_multiples=args["mxl"],
                                        callback=args["cb"], **args["sdxl"]).images)

      image = [imm for im in image for imm in im]

      if(args["ip_adap_en"]):


        ima_temp = []
        for ima in image:
          ima_temp.append(args["model_face"].generate(prompt=pr[0], negative_prompt=negPr[0], image=ima, strength=args["ip_adap_strength_ip"],
                                         faceid_embeds=args["ip_adap_faceid_embeds"], face_image=args["ip_adap_face_image"],
                                         shortcut=args["ip_adap_v2"], s_scale=args["ip_adap_s_scale"], scale=args["ip_adap_scale"],
                                         num_samples=args["batch_size"], width=width, height=height,
                                         guidance_scale=args["strength"], noise=args["ip_adap_noise"], cross_attention_kwargs={"scale": 1.0},
                                         num_inference_steps=args["ip_adap_steps"], seed=seeds[-1], callback=args["cbf"], callback_steps=1)[0])

        ImaTot = ImaTot + ima_temp
      else:
        ImaTot = ImaTot + image

      for i,_ in enumerate(ImaTot):

          metadata = PngInfo()
          metadata.add_text("positive_prompt",  (''.join(pr)))
          metadata.add_text("negative_prompt", (''.join(negPr)))
          metadata.add_text("seed", str(seeds[len(seeds)-i-1]))
          metadata.add_text("gs", str(args["strength"]))
          metadata.add_text("step", str([args["step"]][i%(len([args["step"]]))]))
          metadata.add_text("width", str(args["width"]))
          metadata.add_text("height", str(args["height"]))


          ImaTot[i].save("NewPath.png", pnginfo=metadata)

          ImaTot[i] = Image.open("NewPath.png")

          !rm "./NewPath.png"
  gc.collect()
  torch.cuda.empty_cache()
  return ImaTot


def deepp_ins(step, pred_original_sample, pipe):
    global hbox, vae_tiny, out_preview
    with torch.no_grad():

        if half_value:
          pred_original_sample = pred_original_sample.half()

        imageT = vae_tiny.decode(pred_original_sample).sample

        imageT = (imageT / 2 + 0.5).clamp(0, 1)

        # # we always cast to float32 as this does not cause significant overhead and is compatible with bfloa16
        imageT1 = imageT.cpu().permute(0, 2, 3, 1).float().numpy()

        # # convert to PIL Images
        image = pipe.numpy_to_pil(imageT1)
        # # do something with the Images
        with out_preview:
             clear_output(wait=True)
             for img in image:
                 display(img)
        hbox.children = [out_preview]
        del image
        del imageT
        del imageT1
        gc.collect()
        fresh()
        return

def deepp_ins_lat(step, latent, pipe):
    global hbox, vae_tiny, out_preview
    with torch.no_grad():


        imageT = vae_tiny.decode(latent).sample


        if half_value:
          imageT = imageT.half()

        imageT = (imageT / 2 + 0.5).clamp(0, 1)

        # # we always cast to float32 as this does not cause significant overhead and is compatible with bfloa16
        imageT1 = imageT.cpu().permute(0, 2, 3, 1).float().numpy()

        # # convert to PIL Images
        image = pipe.numpy_to_pil(imageT1)
        # # do something with the Images
        with out_preview:
             clear_output(wait=True)
             for img in image:
                 if img.width>1025 or img.height>1025:
                  w,h = img.size

                  r = w/h
                  w1,h1=(w,h)
                  if img.width>img.height:
                    w1=1024
                    h1=w1/r
                  else:
                    h1=1024
                    w1=r*h1
                  img  = img.resize((int(w1),int(h1)))
                 display(img)
        hbox.children = [out_preview]
        del image
        del imageT
        del imageT1
        gc.collect()
        fresh()
        return

def cb(step, t, latents, pred_original_sample):
    global preview, last_thread

    if "enable_freeU" in locals():
        if strength_freeU < 1 and step >= int(step * denoise * strength_freeU):
                pipe["model"].unet.freeu.ones()
    if preview and ((step+1)%10==0 or step==0):
        if last_thread != None:
            last_thread.exit()
            del last_thread
            last_thread = None
        last_thread = threading.Thread(target=deepp_ins, args=(step, pred_original_sample, pipe["model"])).start()

def cbl(step, t, latents):
    global preview, last_thread

    if "enable_freeU" in locals():
        if strength_freeU < 1 and step >= int(step * denoise * strength_freeU):
                pipe["model"].unet.freeu.ones()
    if preview and ((step+1)%10==0 or step==0):
        if last_thread != None:
            last_thread.exit()
            del last_thread
            last_thread = None
        last_thread = threading.Thread(target=deepp_ins_lat, args=(step, latents, pipe["model"])).start()

In [ ]:
pipe= {"model":None, "clip":None, "clip_lora":None, "model_path":None, "model_config":None, "model_vae":None, "model_face":None}

#Generator

In [ ]:
#@title Path select or Refresh models path
model_path = "/models/checkpoints" #@param {type:"string"}
lora_path = "/models/loras"#@param {type:"string"}
embedding_path = "/models/embeddings"#@param {type:"string"}
vaes_path = "/models/vae"#@param {type:"string"}
ipAd_path = "/models/ip_ada/"#@param {type:"string"}


model_path = COMF +"/"+model_path
lora_path = COMF +"/"+lora_path
embedding_path = COMF +"/"+embedding_path
vaes_path = COMF +"/"+vaes_path
ipAd_path = COMF +"/"+ipAd_path

from pathlib import Path

if not(os.path.exists(model_path)):
  Path(model_path).mkdir(parents=True, exist_ok=True)
if not(os.path.exists(lora_path)):
  Path(lora_path).mkdir(parents=True, exist_ok=True)
if not(os.path.exists(embedding_path)):
  Path(embedding_path).mkdir(parents=True, exist_ok=True)
if not(os.path.exists(vaes_path)):
  Path(vaes_path).mkdir(parents=True, exist_ok=True)
if not(os.path.exists(vaes_path)):
  Path(vaes_path).mkdir(parents=True, exist_ok=True)

models = [i for i in os.listdir(model_path) if i.endswith("safetensors") or i.endswith("ckpt")]
loras = getLoras(lora_path)
embedding = [i for i in os.listdir(embedding_path) if i.endswith("safetensors") or i.endswith("ckpt") or i.endswith(".pt")]
configs = [i for i in os.listdir(model_path) if i.endswith("yaml")]
vaes = [i for i in os.listdir(vaes_path) if i.endswith("safetensors")]
# ipAd = [i for i in os.listdir(ipAd_path) if i.endswith("safetensors") or i.endswith(".pt")]
controlnets = ["canny", "openpose","depth","lineart","normalbae"]

models.sort()
embedding.sort()
configs.sort()
vaes.sort()
# ipAd.sort()
configs = ["default"]+configs
vaes = ["default"]+vaes

In [ ]:
#@title Download model
import subprocess

name = "" #@param {type:"string"}

url= '' #@param{type:'string'}

tipo = "Checkpoint" #@param ["Lora","Checkpoint","Embedding","Vae"]
token = "" #@param{type:"string"}
url += ("&token="+token if len(token)>0 else "")

if "Lora" in tipo and url != "":

  !/usr/bin/wget -c {"\""+url+"\""} -O {"\'"+lora_path+'/'+name+".safetensors\'"}
if "Checkpoint" in tipo and url != "":
  !wget -P -c {"\""+url+"\""} -O {"\'"+model_path+'/'+name+".safetensors\'"}

if "Embedding" in tipo and url != "":
  !wget -P -c {"\""+url+"\""} -O {"\'"+embedding_path+'/'+name+".safetensors\'"}


if "Vae" in tipo and url != "":
  !wget -P -c {"\""+url+"\""} -O {"\'"+vaes_path+'/'+name+".safetensors\'"}

In [ ]:
#@title Model Selector
import copy
import importlib
importlib.reload(lpw_stable_diffusion)
importlib.reload(lpw_stable_diffusion_xl)

models_pick = widgets.Dropdown(options=models)
model_label = widgets.Label("Model:")


config_pick  = widgets.Dropdown(options=configs)
config_label = widgets.Label("Config:")

vae_pick  = widgets.Dropdown(options=vaes)
vae_label = widgets.Label("Vae:")
button = widgets.Button(description="Load", layout=widgets.Layout(width='200px'))


def f(a,b,c):
    print((a,b,c))

ui = widgets.VBox([widgets.HBox([model_label,widgets.Box(layout=widgets.Layout(display='flex',flex_flow='row',align_items='stretch',width='2px')), models_pick]),
                   widgets.HBox([config_label,config_pick]),
                   widgets.HBox([vae_label,widgets.Box(layout=widgets.Layout(display='flex',flex_flow='row',align_items='stretch',width='15px'))   ,vae_pick]),
                   widgets.HBox([widgets.Box(layout=widgets.Layout(display='flex',flex_flow='row',align_items='stretch',width='50px')), button], layout=widgets.Layout(display='flex',flex_flow='row',align_items='center', width="300px"))])
out = widgets.interactive_output(f, {'a': models_pick, 'b':config_pick, 'c':vae_pick})


def loadModel(pipe,model_path, configuration, vae_path):
    global vae_tiny, sdxl
    device = "cuda"
    fresh()
    fresh()
    fresh()
    if "model" in pipe.keys():
      if pipe["model"] != None:
        del pipe["model"]
        if "lora" in pipe.keys():
          del pipe["lora"]
        del pipe
        for _ in range(0,10):
         fresh()
        pipe= {"model":None, "clip":None, "clip_lora":None, "model_path":None, "model_config":None, "model_vae":None, "model_face":None}
        for _ in range(0,10):
          fresh()
        return -1
    if "vae_tiny" in locals() or "vae_tiny" in globals():
      del vae_tiny

    fresh()
    fresh()
    fresh()
    pipe["model_path"]=model_path
    pipe["model_config"]=configuration
    pipe["model_vae"]=vae_path
    s=SafeTensorsFile.open_file(model_path)
    js=s.get_header()
    sdxl = False
    test=False

    if "__metadata__" in str(js) and str(js["__metadata__"])!="{}":
      sdxl = "pony" in str(js["__metadata__"]).lower() or ("modelspec.architecture" in str(js["__metadata__"]) and str(js["__metadata__"]["modelspec.architecture"])=="stable-diffusion-xl-v1-base")
    else:
      if "conditioner.embedders.0" in str(js) or "conditioner.embedders.1" in str(js):
        sdxl=True
    print("sdxl:", sdxl)

    try:
      if sdxl:
        pipe["model"] = lpw_stable_diffusion_xl.SDXLLongPromptWeightingPipeline.from_single_file(model_path,use_safetensors=True, torch_dtype=torch.float16 if half_value else torch.float32, original_config_file=configuration)
        pipe["model"].vae.enable_vae_slicing = True

      else:
        pipe["model"] = lpw_stable_diffusion.StableDiffusionLongPromptWeightingPipeline.from_single_file(model_path,use_safetensors=True, torch_dtype=torch.float16 if half_value else torch.float32, original_config_file=configuration)
    except:
      if sdxl:
        sdxl = False
        pipe["model"] = lpw_stable_diffusion.StableDiffusionLongPromptWeightingPipeline.from_single_file(model_path,use_safetensors=True, torch_dtype=torch.float16 if half_value else torch.float32, original_config_file=configuration)

      else:
        sdxl = True
        pipe["model"] = lpw_stable_diffusion_xl.SDXLLongPromptWeightingPipeline.from_single_file(model_path,use_safetensors=True, torch_dtype=torch.float16 if half_value else torch.float32, original_config_file=configuration)
        pipe["model"].vae.enable_vae_slicing = True

    if not("default" in vae_path):
      pipe["model"].vae = AutoencoderKL.from_single_file(vae_path,use_safetensors=True, torch_dtype=torch.float16 if half_value else torch.float32)
    pipe["model"].enable_vae_tiling()
    pipe["model"].unet.set_attn_processor(AttnProcessor2_0())
    pipe["model"].unet.to(memory_format=torch.channels_last)

    pipe["model"] = pipe["model"].to(device)

    def dummy(images, **kwargs): return images, [False for x in images]
    pipe["model"].safety_checker = dummy
    #pipe["model"].enable_xformers_memory_efficient_attention()
    for _ in range(0,20):
      fresh()
    print("loaded "+model_path.split("/")[-1])
    #pipe["clip"] = copy.deepcopy(pipe["model"].text_encoder.text_model.encoder.layers)
    return 0

def on_button_clicked_ck(b,pipe):
    global vae_tiny
    button.disabled=True
    print("Starting load, please wait")

    configuration = None

    if config_pick.value=="default":
      configuration = None
    else:
      configuration = path(config_pick.value)

    r = loadModel(pipe ,path(models_pick.value), configuration, path(vae_pick.value,ck=False, vae=True))
    if r==-1:
        loadModel(pipe ,path(models_pick.value), configuration, path(vae_pick.value,ck=False, vae=True))

    button.disabled=False

button.on_click(lambda b: on_button_clicked_ck(b,pipe))

display(ui, out)

In [ ]:
#@title Lora Select
from pathlib import Path

pipe["model"].enable_lora()

def f(a):
  print((a))

lora_pick = widgets.Dropdown(options=loras)
button1 = widgets.Button(description="Load")
button2 = widgets.Button(description="Unload")
button3 = widgets.Button(description="Norm W")
strengt = widgets.BoundedFloatText(
    value=1.0,
    min=-6.0,
    max=6.0,
    step=0.05,
    description='Strength:',
    disabled=False
)
ui = widgets.VBox([widgets.HBox([lora_pick]), widgets.HBox([strengt]), widgets.HBox([ button1, button2, button3])])
out = widgets.interactive_output(f, {'a': lora_pick})
lora_pick.ok=0

if "lora" in pipe.keys():
  print("Active lora: ",pipe["model"].get_active_adapters(), pipe["lora"])

def getNames(v=0):

  return [l if v==0 else ln for l,ln,st in pipe["lora"]]

def getStrengt():
  return [float(st) for l,ln,st in pipe["lora"]]

def on_button_clicked(b):
    global pipe
    button1.disabled=True
    button2.disabled=True
    button3.disabled=True
    device = "cuda"

    if "lora" not in pipe.keys():
      pipe["lora"]=[]


    path_loraName = str(Path(lora_path+"/"+lora_pick.value["name_file"]))
    w_name = str(lora_pick.value["name_file"]).replace("-","").replace("_","")
    ad_name = w_name.split(".")[0]
    #print(path_loraName, w_name, ad_name)

    if(ad_name not in getNames()):
      pipe["model"].load_lora_weights(path_loraName, weight_name=w_name, adapter_name=ad_name)

    print("loaded "+(path_loraName)+" strengt "+str(strengt.value))
    print("tokens: "+str(get_tokens(lora_pick.value["path"]+"/"+lora_pick.value["name_file"])))
    if(ad_name not in getNames()):
      last_name= pipe["model"].get_active_adapters()[-1]
      pipe["lora"].append([ad_name,last_name,float(strengt.value)])
    else:
      el = [[l,ln,s] for l,ln,s in pipe["lora"] if l==ad_name][0]
      pipe["lora"][pipe["lora"].index(el)] = [el[0],el[1],float(strengt.value)]
    print(getNames(),getStrengt())

    pipe["model"].set_adapters(getNames(1),adapter_weights=getStrengt())

    print("Active lora: ",pipe["model"].get_active_adapters(), pipe["lora"])
    button1.disabled=False
    button2.disabled=False
    button3.disabled=False

def on_button_clicked2(b):
    global pipe
    button2.disabled=True
    button1.disabled=True
    button3.disabled=True

    if "lora" not in pipe.keys():
      pipe["lora"]=[]

    if lora_pick.value["name_file"].split(".")[0] in getNames():
      del pipe["lora"][pipe["lora"].index([[l,s] for l,s in pipe["lora"] if l==lora_pick.value["name_file"].split(".")[0]][0])]
      pipe["model"].set_adapters(getNames(),adapter_weights=getStrengt())
      print("Lora unloaded ")

    button1.disabled=False
    button2.disabled=False
    button3.disabled=False

def normalize(b):
  global pipe
  nor = sum(getStrengt())
  nStrengt = []
  for i,(l,s) in enumerate(pipe["lora"]):
    nStrengt.append(s/nor)
    pipe["lora"][i] = [l,s/nor]
  pipe["model"].set_adapters(getNames(), nStrengt)

  print("Active lora: ", pipe["lora"])

button1.on_click(lambda b: on_button_clicked(b))
button2.on_click(lambda b: on_button_clicked2(b))
button3.on_click(lambda b: normalize(b))

last_change = {}
def on_change(change):
    global last_change

    if "lora" not in pipe.keys():
      pipe["lora"]=[]
    if change.name=="index":
      print("Active lora: ",pipe["model"].get_active_adapters(), pipe["lora"])
    def nothing(c):
        pass
    def isFolder(val):
       return not("name_file" in val.keys())

    if change['type'] == 'change' and change['name'] == 'value' and lora_pick.ok==0:

        lora_pick.ok=1


        if ".." not in lora_pick.options and isFolder(lora_pick.value):

            new_dict = {"..":{}}
            new_dict.update(change.new)
            lora_pick.options = new_dict
            lora_pick.index = 1

            lora_pick.ok=2


    elif change['type'] == 'change' and change['name'] == 'value' and lora_pick.value=={} and lora_pick.ok==2:
        lora_pick.ok=3
        lora_pick.observe(nothing, names=["value"])
        lora_pick.options = loras

        lora_pick.observe(on_change, names=["value"])

        lora_pick.ok=0


lora_pick.observe(on_change)

display(ui, out)

In [ ]:

#@title Embedding Select

def f(a):
  print((a))

emb_pick = widgets.Dropdown(options=embedding)
button13 = widgets.Button(description="Load")
strengt1 = widgets.BoundedFloatText(
    value=1.0,
    min=0.0,
    max=2.0,
    step=0.05,
    description='Strength:',
    disabled=False
)
ui = widgets.VBox([widgets.HBox([emb_pick]), widgets.HBox([ button13])])
out = widgets.interactive_output(f, {'a': emb_pick})

def butt(b,pipe):
  pipe["model"].load_textual_inversion(embedding_path, weight_name=emb_pick.value, local_files_only=True)

button13.on_click(lambda b:butt(b,pipe))
display(ui,out)

In [ ]:
#@title Clip Skip
#@markdown work with base model and lora on base model
clip_skip = 2 #@param {type:"integer"}

try:
  if "clip_layers" in locals():
    del clip_layers
finally:
  print()

clip_layers = None

if "clip_lora" in pipe.keys() and pipe["clip_lora"]!=None:
  clip_layers = pipe["clip_lora"]
else:
  clip_layers = pipe["clip"]
if clip_skip > 0:
  del pipe["model"].text_encoder.text_model.encoder.layers
  pipe["model"].text_encoder.text_model.encoder.layers = copy.deepcopy(clip_layers[:-clip_skip])
else:
  del pipe["model"].text_encoder.text_model.encoder.layers
  pipe["model"].text_encoder.text_model.encoder.layers = copy.deepcopy(clip_layers)

## ControlNet

In [ ]:
#@title ControlNet


def f(a):
  print((a))

def rename(val):
    if "depth" in val:
        return "control_v11f1p_sd15_depth"
    else:
        return "control_v11p_sd15_"+val

cnt_pick = widgets.Dropdown(options=controlnets)
button14 = widgets.Button(description="Load")
strengt15 = widgets.BoundedFloatText(
    value=1.0,
    min=0.0,
    max=2.0,
    step=0.05,
    description='Strength:',
    disabled=False
)
ui = widgets.VBox([widgets.HBox([cnt_pick]), widgets.HBox([ button14])])
out = widgets.interactive_output(f, {'a': cnt_pick})

def butt(b,pipe):
  controlnet = lpw_stable_diffusion.ControlNetModel.from_pretrained("lllyasviel/"+rename(cnt_pick.value), torch_dtype=torch.float16)
  pipe["model"].set_controlnet(controlnet)

button14.on_click(lambda b:butt(b,pipe))
display(ui)

In [ ]:
#@title #Load image (ControlNet)
from google.colab import files
import ipywidgets as widgets
from IPython.display import display
import PIL
from PIL import Image, ImageDraw, ImageFilter

loadImage = False
image_controlnet = None


display("Load Image:")
Up = files.upload()
for k, v in Up.items():
    image_controlnet=Image.open(k)

image_controlnet.size

In [ ]:
#@title Preprocess ControlNet
import cv2
import numpy


precprocess = "openpose_full" #@param ["canny", "openpose", "openpose_full", "normal_bae", "depth_leres", "depth_leres++"]
size_precision = 768 #@param [512, 640, 768, 960]

preprocessor =  Processor(precprocess)
open_cv_image = numpy.array(image_controlnet)
# Convert RGB to BGR
open_cv_image = open_cv_image[:, :, ::-1].copy()
image_controlnet = preprocessor(open_cv_image, detect_resolution=size_precision)
if "canny" in precprocess:
  image_controlnet=Image.fromarray(image_controlnet)

image_controlnet

##Image2Image

In [ ]:
#@title #Load image (img2img)
from google.colab import files
import ipywidgets as widgets
from IPython.display import display
import PIL
from PIL import Image, ImageDraw, ImageFilter

#@markdown #Load image from upload file
#@markdown Size of image

#@title Dimensioni
def sizeR(size):
  if size%64<=16:
    return size
  else:
    return (size//64)*64+64

loadImage = False
mask = Image.new('RGBA', (512, 512),  (255, 255, 255))
initt = None
url = None


display("Load Image:")
Up = files.upload()
for k, v in Up.items():
    initt=Image.open(k)
display("Load Mask:")
Up = files.upload()
for k, v in Up.items():
      mask=Image.open(k)


print(initt.size)

In [ ]:
#@title Resize image


width = 576 #@param Integer
height = 1280 #@param Integer
mSize = width

rat = mSize

initt.thumbnail((rat,rat))
w,h= (width,height)

w = sizeR(w)
h = sizeR(h)
initt=initt.resize((w,h))
mask = mask.resize((w,h))

display(image_grid([initt,mask],1,2))

def loadFromUrl():
  url="https://www.google.com/images/branding/googlelogo/2x/googlelogo_light_color_272x92dp.png"
  response = requests.get(url)
  initt = Image.open(BytesIO(response.content))

In [ ]:
#@title Drawing mask
#@markdown Only for PC
from IPython.display import HTML
from google.colab.output import eval_js
import PIL.ImageOps
from base64 import b64decode
from base64 import b64encode

canvas_html = """
<img src='%s' width=%d  height=%d' >
<canvas width=%d height=%d style='position:absolute; top:0; left:0; z-index:2'></canvas>

<button>Finish</button>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')
ctx.lineWidth = %d
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}
canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}
var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})
</script>
"""



def draw(filename='drawing.png', w=400, h=200, line_width=1):

  mime_type = None
  initt.save("/content/my_image.png")
  path_to_image = "/content/my_image.png"

  mime_type = 'image/png'

  img = open(path_to_image, 'rb').read()
  data_url = 'data:image/jpeg;base64,' + b64encode(img).decode()

  display(HTML(canvas_html % (data_url, w, h, w, h, line_width)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)

  return len(binary)


w1,h1 = initt.size
draw("my_drawing.png",w1,h1,min(w1,h1)/10)
mask = Image.open("my_drawing.png")

newImage = []
for item in mask.getdata():
    if item[3] <=10:
        newImage.append((0,0,0, 255))
    else:
        newImage.append((255,255,255,255))

mask.putdata(newImage)

display(mask)

In [ ]:
#@title prepare mask
#@markdown lunch to synchronize the image
import PIL.ImageOps



if mask.mode == 'RGBA':
    r,g,b,a = mask.split()
    rgb_image = Image.merge('RGB', (r,g,b))

    mask = Image.merge('RGB', (r,g,b))


if initt.mode == 'RGBA':
    r1,g1,b1,a1 = initt.split()
    rgb_image = Image.merge('RGB', (r1,g1,b1))


    initt = Image.merge('RGB', (r1,g1,b1))
    print("cl")

mask, initt

#Inference

In [ ]:
#@title Clean Memory
#@markdown There's a CUDA Memory error?</br> LUNCH IT !!!
vae_tiny=None
for _ in range(0,20):
  fresh()

In [ ]:
#@title generate


if "args" in locals():
    del args
if "imagess" in locals():
    del imagess
if "images" in locals():
    del images
if "out_preview" in locals():
    clear_output()
    del out_preview
if not("vae_tiny" in locals()) or vae_tiny == None:
    vae_load()
if "strength_freeU" in locals() and strength_freeU>0:
    pipe["model"].unet.freeu.sd21()

gc.collect()

args = {}

preview = True #@param {type:"boolean"}
prompt = "Masterpiece photograph, a beautiful world in a bottle" #@param {type:"string"}
negPrompt = "horrible, ugly, text, watermark"}
width = 600 #@param Integer
height = 904 #@param Integer
step = 44 #@param Integer
seed = None #@param
batch_size = 1 #@param Integer
batch_number = 1 #@param Integer
scheduler = "EulerAncestralDiscreteScheduler" #@param ['DDIMScheduler', 'DDPMScheduler', 'DEISMultistepScheduler', 'DPMSolverMultistepScheduler', 'DPMSolverSinglestepScheduler', 'EulerAncestralDiscreteScheduler', 'EulerDiscreteScheduler', 'HeunDiscreteScheduler', 'KDPM2AncestralDiscreteScheduler', 'KDPM2DiscreteScheduler', 'LMSDiscreteScheduler', 'PNDMScheduler', 'UniPCMultistepScheduler']
strength = 3.7 #@param {type:"slider", min:0, max:30, step:0.1}
#@markdown ---
#@markdown #HighRes fix
highres_fix = True #@param {type:"boolean"}
scale = 1.75 #@param {type:"slider", min:0, max:2.5, step:0.05}
denoise = 0.555 #@param {type:"slider", min:0, max:1, step:0.005}
strength_hfx = 5.3 #@param {type:"slider", min:0, max:30, step:0.1}
seed_hfx = None #@param
step_hfx = 60 #@param Integer
#@markdown ---
#@markdown #Image2Image
image2image = False #@param {type:"boolean"}
inpainting = False #@param {type:"boolean"}
imageStrength = 1 #@param {type:"slider", min:0, max:1, step:0.01}
imageBlur = False #@param {type:"boolean"}
#@markdown ---
#@markdown #ControlNet
controlnet = False #@param {type:"boolean"}
controlnet_strenght = 0.85 #@param {type:"number"}
start_controlnet = 0.51 #@param {type:"slider", min:0, max:1, step:0.01}
end_controlnet = 0.77 #@param {type:"slider", min:0, max:1, step:0.01}

args['prompt'] = prompt
args["negPrompt"] = negPrompt
args["width"] = width
args["height"] = height
args["step"] = step
args['seed'] = [seed-1] if str(seed)[1:].isnumeric() else [None]
args["batch_size"] = batch_size
args["batch_number"] = batch_number
args["strength"] = strength
args["init_blur"] = None
args["controlnet_strenght"] = controlnet_strenght if controlnet else 0.0
args["controlnet_image"] = image_controlnet if controlnet else None
args["start_cnt"] =  start_controlnet if controlnet else 0.0
args["end_cnt"] = end_controlnet if controlnet else 0.0
args["ip_adap_en"] = ip_adapter
args["sdxl"] = {}

args['imageStrength'] = imageStrength

if args["seed"][0] == None:
  args["seed"] = [random.randint(-int(sys.maxsize),int(sys.maxsize))]
args["cb"] = cb if not(sdxl) else cbl

pipe["model"].scheduler = modifyScheduler(pipe,scheduler)

hbox = widgets.HBox(children=[])
out_preview = widgets.Output()
display(out_preview)

if image2image or inpainting:
  args["init_image"] = initt
  args["init_image"] = args["init_image"].resize((width, height))
  if (imageBlur):
    args['init_blur'] = (initt.filter(ImageFilter.BLUR))
    args['init_image'] = initt
  args['init_blur'] = initt
  args["mask_image"] = mask.convert("RGBA")

  args['init_image'] = args['init_image'].resize((width, height))
  args['init_blur'] =  args['init_blur'].resize((width, height))
  args['mask_image'] = args['mask_image'].resize((width, height))
else:
  args['init_image'] = Image.new('RGBA', (width, height),  (255, 255, 255))
  args['init_blur'] = Image.new('RGBA', (width, height),  (255, 255, 255))
  args['mask_image'] = Image.new('RGBA', (width, height),  (255, 255, 255))
  args['imageStrength'] = 1.0

if sdxl:
  args["sdxl"] = {"num_images_per_prompt":batch_size}


args["mxl"] = int(min(max(len(args["prompt"]), len(args["negPrompt"]))/30, 77))
args["scale_lora"] = 1.0
args["Pp"] = "image" if image2image else ("inpaint" if inpainting else "text")

fresh()
fresh()
fresh()
fresh()

images = generate(pipe["model"],args)



imagess=[]
if highres_fix:
  args["step"] = step_hfx
  args["ip_adap_en"]=ip_adapter_highfix_pass
  args["controlnet_strenght"] = 0.0
  args["controlnet_image"] =  None
  args["strength"] = strength_hfx
  for ima in images:
    fresh()
    o_w = int(ima.size[0]*scale)
    o_h = int(ima.size[1]*scale)
    args["init_image"] = ima.resize((o_w,o_h))
    args["width"]=o_w
    args["height"]=o_h
    args['imageStrength'] = denoise
    args["batch_size"] = 1
    args["batch_number"] = 1
    if sdxl:
      args["sdxl"] = {"num_images_per_prompt":1}

    if seed_hfx == None:
      args["seed"] = [random.randint(-int(sys.maxsize),int(sys.maxsize))]
    else:
      args['seed'] = [seed_hfx]
    args["Pp"] = "image"
    fresh()
    fresh()
    imagess.append(generate(pipe["model"],args)[0])
  images=imagess


with out_preview:
    clear_output(wait=True)
    for img in images:
        display(img)
del out_preview
del hbox
fresh()
fresh()
fresh()
fresh()

In [ ]:
#@title Display images when connection loss
for im in images:
  display(im)

In [ ]:
#@title Save image
x= [X for X in os.listdir("/content/") if "IMG" in X]
x.sort()
images[0].save("/content/IMG"+str(len(x))+".png")

In [ ]:
#@title Copy images to drive
if not(os.path.exists("/content/drive/MyDrive/imgs/")):
  os.mkdir("/content/drive/MyDrive/imgs/")
for e in  [X for X in os.listdir("/content/") if "IMG" in X]:
  print(e)
  !cp "{"/content/"+e}" "{"/content/drive/MyDrive/imgs/lol"+e}"

In [ ]:
#@title ADetailer

#@markdown A first implementation of ADetailer, width and height will be the render size of pipe.</br> The pixel_increase_millimesimal increases the face frame as a millimesimal of the image size.</br> I raccomend to use a positive prompt with "face, highly detailed face, character..."

import importlib
importlib.reload(ADetailer)

preview = False

model = "face_yolov8n_v2.pt" #@param ["face_yolov8n_v2.pt","face_yolov8m.pt","hand_yolov8n.pt"]
positive = "Looking to viewer, highly detailed face, $PROMPT" #@param {type:"string"}
negative = "Deformed face, red, $NPROMPT" #@param {type:"string"}
step = 40 #@param {type:"integer"}
width = 960 #@param {type:"integer"}
height = 960 #@param {type:"integer"}
pixel_increase_millimesimal = 250 #@param {type:"integer", min :0, max:100}
blur_radius = 10 #@param {type:"integer", min :0, max:100}
denoise = 0.25 #@param {type:"slider", min:0, max:1.0, step:0.01}
dilatation = 10 #@param {type:"integer", min:0, max:100.0, step:1}

cfg =  8.1 #@param {type:"slider", min:0, max:30.0, step:0.05}

#@markdown Reapeat recursively the diffusion every time the denoise decrease in function of decrease function
recursively = 2 #@param {type:"integer", min:0, max:20}

decrease_function = "linear(x)->x-x/20" #@param ["constant(x)->x","linear(x)->x-x/20","linear(x)->x-x/10","linear(x)->x-x/5","linear(x)->x-x/2"]

def decrease(x):
  x_n = x
  if "linear" in decrease_function:
    number = int(decrease_function.split("/")[1])
    x_n = x - x/number
  return x_n

args1 = {}

args1["step"] = step
args1["denoise"] = denoise
args1["dilatation"] = dilatation
args1["cfg"] = cfg
args1["positive"] = "highly detailed face, " + args["prompt"] if positive=="" else (positive.replace("$PROMPT",args["prompt"]) if "$PROMPT" in positive else positive)
args1["negative"] = args["negPrompt"] if negative=="" else (negative.replace("$NPROMPT",args["negPrompt"]) if "$NPROMPT" in negative else negative)
args1["width"] = width
args1["height"] = height
args1["pipe"] = pipe["model"]
args1["percent"] = pixel_increase_millimesimal
args1["blur_radius"] = blur_radius
args1["ip_adap_en"] = ip_adapter

if "cb" in args.keys():
  args1["cb"] = args["cb"]
if "detailer" in locals():
  del detailer

detailer = ADetailer.ADetailerDiffuser(model, 0.25)
images1= [im.copy() for im in images]
for i in range(0,recursively):
  print(str(i)+"/"+str(recursively)+" iteration ("+str(args1["denoise"])+")")
  imagess = []
  for img in images1:
    if not(ip_adapter):
      args1["pipe"].enable_vae_slicing()
    imagess.append(detailer(img, args1))

  for im in (imagess):
    display(im)
    im.show()
  fresh()
  images1 = [ima.copy() for ima in imagess]
  args1["denoise"] = decrease(args1["denoise"])

In [ ]:
#@title Save image
x= [X for X in os.listdir("/content/") if "IMGD" in X]
x.sort()
images1[0].save("/content/IMGD"+str(len(x))+".png")